In [1]:
import numpy as np
import pandas
import pm4py
from matplotlib import pyplot as plt
from sklearn.mixture import GaussianMixture
import scipy.stats as stats
import ot
import os
from tqdm import tqdm
import collections
import matplotlib.dates as md
import importlib
import pickle
import random
import math
import CRPS.CRPS as pscore
import datetime

pandas.set_option('display.max_columns', None)
#pandas.set_option('display.max_rows', None)


import sys
sys.path.append('../../TaskExecutionTimeMining/')
from drbart_parser import *
from event_log_transformer import *

#sys.path.append('../../Evaluation')
sys.path.append('../../Evaluation/')
from normal_evaluation.drbart_evaluation import *

In [2]:
with open('../transformed_event_logs/artificial_start_end_2_train.pickle', 'rb') as f:
    train_data = pickle.load(f)

In [3]:
train_data

,concept:name,lifecycle:transition_start,time:timestamp_start,org:resource,case:concept:name,id_start,lifecycle:transition_complete,time:timestamp_complete,id_complete,duration,duration_seconds,seconds_in_day,DIAGNOSIS,QUALITY_CONTROL,REPAIR,1,Clark,Jane,Joe,Karsten
0,DIAGNOSIS,START,2020-01-01 03:57:40.044121+00:00,Jane,0,1,COMPLETE,2020-01-01 04:34:23.549454+00:00,2,0 days 00:36:43.505333,2203,14260,1,0,0,0,0,1,0,0
1,REPAIR,START,2020-01-01 04:34:23.549454+00:00,Joe,0,4,COMPLETE,2020-01-01 14:30:27.423999+00:00,5,0 days 09:56:03.874545,35763,16463,1,0,1,0,0,1,1,0
3,DIAGNOSIS,START,2020-01-01 08:16:35.844753+00:00,Jane,1,10,COMPLETE,2020-01-01 08:47:14.772217+00:00,11,0 days 00:30:38.927464,1838,29795,1,0,0,0,0,1,0,0
4,REPAIR,START,2020-01-01 08:47:14.772217+00:00,Karsten,1,13,COMPLETE,2020-01-01 13:27:22.316694+00:00,14,0 days 04:40:07.544477,16807,31634,1,0,1,0,0,1,0,1
5,QUALITY_CONTROL,START,2020-01-01 13:27:22.316694+00:00,Clark,1,16,COMPLETE,2020-01-01 17:21:42.825766+00:00,17,0 days 03:54:20.509072,14060,48442,1,1,1,0,1,1,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5400,DIAGNOSIS,START,2024-12-26 18:05:59.540931+00:00,Clark,1800,16201,COMPLETE,2024-12-26 19:13:26.892809+00:00,16202,0 days 01:07:27.351878,4047,65159,1,0,0,0,1,0,0,0
5399,QUALITY_CONTROL,START,2024-12-26 19:03:41.622915+00:00,Joe,1799,16198,COMPLETE,2024-12-26 21:39:17.312456+00:00,16199,0 days 02:35:35.689541,9335,68621,1,1,1,1,0,0,1,1
5401,REPAIR,START,2024-12-26 19:13:26.892809+00:00,Clark,1800,16204,COMPLETE,2024-12-27 08:23:46.946760+00:00,16205,0 days 13:10:20.053951,47420,69206,1,0,1,0,2,0,0,0
5402,QUALITY_CONTROL,START,2024-12-27 08:23:46.946760+00:00,Jane,1800,16207,COMPLETE,2024-12-27 11:10:44.009123+00:00,16208,0 days 02:46:57.062363,10017,30226,1,1,1,0,2,1,0,0


In [4]:
from dumas_model import DumasModel

dm = DumasModel(train_data, resource=True,
               concept_name = 'concept:name')
dm.set_up_models()

with open('./dumas_model.pickle', 'wb') as handle:
    pickle.dump(dm, handle)

In [5]:
dm = DumasModel(train_data, resource=False,
               concept_name = 'concept:name')
dm.set_up_models()

with open('./dumas_model_no_resource.pickle', 'wb') as handle:
    pickle.dump(dm, handle)